## Chapter 7. Ensemble Learning and Random Forests

A group of predictors is called an _ensemble_; thus, this technique is called _Ensemble Learning_, and an
Ensemble Learning algorithm is called an _Ensemble method_.

_Random Forest_: train a group of Decision Tree classifiers, each on a different random subset of the
training set. To make predictions, you just obtain the predictions of all individual trees, then predict the
class that gets the most votes.

You will often use Ensemble methods near the end of a project, once you have already built a few good predictors, to combine them into an even better predictor.

### Voting Classifiers

_Hard voting_ classifier: aggregate the predictions of each classifier and predict the class that gets the most votes.

Somewhat surprisingly, this voting classifier often achieves a higher accuracy than the best classifier in
the ensemble. In fact, even if each classifier is a _weak learner_ (meaning it does only slightly better than
random guessing), the ensemble can still be a _strong learner_ (achieving high accuracy), provided there
are a sufficient number of weak learners and they are sufficiently diverse.

<font color=blue>_TIP_</font>
>Ensemble methods work best when the predictors are as independent from one another as possible. One way to get diverse classifiers is to train them using very different algorithms. This increases the chance that they will make very different types of errors, improving the ensemble's accuracy.

_Soft voting_: If all classifiers are able to estimate class probabilities, then predict the class with the highest class probability, averaged over all the individual classifiers. (It often achieves higher performance than hard voting because it gives more weight to highly confident votes.)

### Bagging and Pasting

Another approach is to use the same training algorithm for every predictor, but to train them on different random subsets of the training set. When sampling is performed with replacement, this method is called _bagging_ (short for bootstrap aggregating). When sampling is performed without replacement, it is called _pasting_.




